In [1]:
#Importing necessary modules 
import random
from deap import base, creator, tools, algorithms
import numpy
import math

In [2]:
# Empezamos por declarar el fitness de cada individuo: Si es un problema
# de maximización el valor del atributo weights es 1, si por el contrario
# el problema es de minimización el valor es -1.
# Consideramos que nuestro objetivo es reducir al mínimo el número de errores
# que contenga el circuito.


In [3]:
creator.create('Fitness', base.Fitness, weights=(-1.0,))

In [4]:
#Consideramos los individuos del problema como las puertas lógicas.

In [5]:
creator.create('Individuo', list, fitness = creator.Fitness)

In [6]:
# A continuación debemos de crear la caja de herramienta donde vamos a
# registrar los elementos necesarios para poder aplicar el algoritmo genético.

In [7]:
caja_de_herramientas = base.Toolbox()

In [8]:
#Declaramos las puertas lógicas
def puerta_or(a,b):
    return a+b

def puerta_and(a,b):
    return a*b

def puerta_not(a):
    if a==0:
        return 1
    if a==1:
        return 0

def puerta_nand(a,b):
    if a==0 and b==0:
        return 1
    elif a==0 and b==1:
        return 1
    elif a==1 and b==0:
        return 1
    elif a==1 and b==1:
        return 0

def puerta_xor(a,b):
    if a==1 and b==1:
        return 0
    return a+b

In [9]:
# Registramos una función gen que devuelve un número de 0 a 5, los cuales
# representan las distintas puertas anteriores puestas en orden más la 
# puerta desactivada.

In [10]:
caja_de_herramientas.register('gen', random.randint, 0, 5)

In [11]:
# Vamos a implementar la posibilidad de elegir libremente los valores del
# número de capas y de puertas lógicas

In [12]:
M = int(input("Numero de capas: "))
N = int(input("Numero de puertas: "))
#Semilla para la generación de números aleatorios
print("Circuito Inicial:")
random.seed(12345) 
for _ in range(M*N):
    print(caja_de_herramientas.gen())
    

Numero de capas: 3
Numero de puertas: 4
Circuito Inicial:
3
5
0
2
2
1
2
4
3
1
2
0


In [13]:
# A continuación llamamomos a la funcion individuo que nos genera un individuo
# con sus genes. Las conexiones máximas que puede tener una puerta son 3.

In [14]:
caja_de_herramientas.register('individuo', tools.initRepeat,
                              container=creator.Individuo, func=caja_de_herramientas.gen, n=3*(M*N))

In [15]:
random.seed(12345)
caja_de_herramientas.individuo()
# El primer gen se asocia al tipo de puerta de la capa, el segundo y el tercero
# se asocia al tipo de puerta del que recibe señal

[3,
 5,
 0,
 2,
 2,
 1,
 2,
 4,
 3,
 1,
 2,
 0,
 3,
 2,
 4,
 5,
 1,
 4,
 4,
 1,
 2,
 5,
 5,
 0,
 4,
 5,
 3,
 4,
 4,
 1,
 1,
 1,
 5,
 0,
 1,
 2]

In [16]:
# Consideramos una población como una lista de 10 individuos

In [17]:
caja_de_herramientas.register('población', tools.initRepeat,
                              container=list, func=caja_de_herramientas.individuo, n=10)

In [18]:
random.seed(12345)
caja_de_herramientas.población()

[[3,
  5,
  0,
  2,
  2,
  1,
  2,
  4,
  3,
  1,
  2,
  0,
  3,
  2,
  4,
  5,
  1,
  4,
  4,
  1,
  2,
  5,
  5,
  0,
  4,
  5,
  3,
  4,
  4,
  1,
  1,
  1,
  5,
  0,
  1,
  2],
 [2,
  0,
  3,
  2,
  0,
  4,
  5,
  5,
  3,
  0,
  0,
  5,
  1,
  4,
  1,
  2,
  0,
  5,
  3,
  0,
  5,
  4,
  1,
  5,
  3,
  1,
  4,
  4,
  5,
  2,
  0,
  1,
  0,
  3,
  1,
  5],
 [4,
  5,
  1,
  2,
  4,
  3,
  4,
  1,
  0,
  1,
  2,
  0,
  1,
  1,
  2,
  4,
  2,
  1,
  3,
  2,
  0,
  0,
  0,
  5,
  3,
  0,
  3,
  1,
  0,
  5,
  2,
  1,
  5,
  4,
  0,
  3],
 [4,
  0,
  0,
  5,
  3,
  2,
  4,
  3,
  4,
  1,
  2,
  5,
  5,
  0,
  4,
  1,
  0,
  3,
  1,
  2,
  1,
  1,
  1,
  1,
  2,
  1,
  4,
  0,
  4,
  5,
  0,
  1,
  0,
  5,
  3,
  1],
 [4,
  0,
  2,
  4,
  3,
  0,
  3,
  3,
  1,
  1,
  1,
  0,
  2,
  2,
  1,
  3,
  1,
  1,
  4,
  5,
  4,
  1,
  3,
  4,
  5,
  3,
  2,
  0,
  0,
  5,
  3,
  5,
  1,
  5,
  4,
  4],
 [1,
  5,
  1,
  4,
  1,
  2,
  4,
  2,
  1,
  4,
  4,
  5,
  5,
  0,
  2,
  3,
  0,
  2,
  4,


In [19]:
# Como cada puerta tiene 2 entradas, el número de valores de entrada es 
# el doble del número de puertas

In [20]:
ValoresEntrada = numpy.empty(N*2, dtype=object)

In [21]:
def decode(individuo, ValoresEntrada):
    Puertas_Logicas=[]
    Salida_Esperada=[]
    Entradas1=[]
    Entradas2=[]
    a = 0
    b = 0
    
    for i in range(0,len(individuo)):
        if(i%3==0):
            Puertas_Logicas.append(individuo[i])
        elif(i%3==1):
            Entradas1.append(individuo[i])
        elif(i%3==2):
            Entradas2.append(individuo[i])
    print(Puertas_Logicas,Entradas1,Entradas2)
    
    #primera capa de 3*3-> 9 genes 3 capas
    individuo[1]=0 #con esto tenemos la primera capa con valores iniciales
    individuo[2]=0
    individuo[4]=0
    individuo[5]=0
    individuo[7]=0
    individuo[8]=0
    
    for i in range(0,9):
        if(i%3==0):
            Puertas_Logicas.append(individuo[i])
        elif(i%3==1):
            a = individuo[i]
        elif(i%3==2):
            b = individuo[i]
    
    Salida_Esperada.append(calculaSalida(a,b,individuo,Puertas_Logicas))
    # por cada capa un bucle y voy metiendo el resultado de la anterior capa 
    # y la otra anterior 
    for i in range(10,19):
        if(i%3==1):
                a=individuo[i] 
        elif(i%3==2):
                b=individuo[i]
                
    Salidas_Capas_Anteriores = [Salida_Esperada[0],Salida_Esperada[1],
                               Salida_Esperada[2],Salida_Esperada[4],
                               Salida_Esperada[5],Salida_Esperada[6]]
    for i in range(20,28): #indice de la puerta es la salida de esa puerta
       # y si hacemos cada dos capas anteriores un array, 
       # y ese array el indice es la posicion relativa de las entradas
       # array de tamaño 6 ahi va la salida de las 3 puertas anteriores 
       #y las otras 3 puertas anteriores 
        if(SalidasCapasAnteriores.append(i)==individuo[i]):
            if(i%3==1):
                    a=Salida_Esperada[i]
            elif(i%3==2):
                    b=Salida_Esperada[i]
            #deberia de devolver de ResultadoSalida 
            #la salida dela ultima capa 
    return(Salida_Esperada,Puertas_Logicas)#puerta 0->indice 0->salida de esa puerta 

In [22]:
#calculo de las salidas de la puerta al tener elvalor de las entradas 
#ahora me hace falta contemplar el tipo de puertas y las conexiones de
#las entradas.

In [23]:
def calculaSalidaNot(a):
    c=operator.not_(a)
    if(c==True):
        c=1
    elif(c==False):
        c=0    
    return c

def calculaSalidaNot(b):
    c=operator.not_(b)
    if(c==True):
        c=1
    elif(c==False):
        c=0    
    return c

In [24]:
#funcion auxiliar calculo salida, tabla de verdad.
def calculaSalida(a, b, individuo,Puertas):
    c=0
    for i in range(0,len(Puertas)):
        if(i%3==0):
            if(i==1):#AND
                c=operator.and_(a, b)#funciona
            elif(i==2):#OR
                c=operator.or_(a, b) #funciona
            elif(i==3):#XOR
                c=operator.xor(a, b)#funciona
            elif(i==4):#NAND
                c=operator.not_(operator.and_(a,b) ) #funciona
                if(c==True):
                    c=1
                elif(c==False):
                    c=0
            elif(i==5):#NOT Funciona
                if(not a == None ):
                    calculaSalidaNot(a)
                elif(not b == None ):
                    calculaSalidaNot(b)
            elif(i==6):#desactivada
                c=0
    return c

In [25]:
# Vamos a registrar los operadores que va a utilizar el algoritmo genético.

In [26]:
caja_de_herramientas.register('mate', tools.cxOnePoint)
caja_de_herramientas.register('mutate', tools.mutFlipBit, indpb=0.1)
# indpb es la probabilidad de mutación de cada gen del cromosoma

In [27]:
random.seed(12345)
caja_de_herramientas.mate([1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
                          [0, 0, 0, 0, 0, 1, 1, 1, 1, 1])

([1, 1, 1, 1, 1, 0, 0, 1, 1, 1], [0, 0, 0, 0, 0, 1, 1, 0, 0, 0])

In [28]:
random.seed(12345)
caja_de_herramientas.mutate([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

([0, 1, 0, 0, 0, 0, 0, 0, 0, 0],)

In [29]:
# Como método de selección de individuos, registramos en la caja de herramientas
# el método de selección por torneo, en el que para seleccionar un individuo 
# se elige al azar una cierta cantidad de individuos y de entre ellos se 
# seleccina el más apto.

In [30]:
caja_de_herramientas.register('select', tools.selTournament, tournsize=3)

In [31]:
random.seed(12345)
P = caja_de_herramientas.población()
caja_de_herramientas.select(P, 5)

[[4,
  0,
  0,
  5,
  3,
  2,
  4,
  3,
  4,
  1,
  2,
  5,
  5,
  0,
  4,
  1,
  0,
  3,
  1,
  2,
  1,
  1,
  1,
  1,
  2,
  1,
  4,
  0,
  4,
  5,
  0,
  1,
  0,
  5,
  3,
  1],
 [4,
  1,
  4,
  4,
  5,
  3,
  4,
  0,
  4,
  1,
  4,
  4,
  2,
  4,
  1,
  3,
  3,
  3,
  0,
  2,
  5,
  0,
  3,
  2,
  5,
  4,
  5,
  4,
  0,
  2,
  3,
  3,
  0,
  2,
  2,
  3],
 [0,
  3,
  1,
  1,
  0,
  1,
  2,
  1,
  2,
  3,
  1,
  0,
  1,
  0,
  4,
  0,
  2,
  3,
  2,
  0,
  1,
  0,
  3,
  1,
  4,
  1,
  2,
  4,
  4,
  2,
  1,
  5,
  2,
  0,
  0,
  0],
 [2,
  0,
  3,
  2,
  0,
  4,
  5,
  5,
  3,
  0,
  0,
  5,
  1,
  4,
  1,
  2,
  0,
  5,
  3,
  0,
  5,
  4,
  1,
  5,
  3,
  1,
  4,
  4,
  5,
  2,
  0,
  1,
  0,
  3,
  1,
  5],
 [2,
  0,
  3,
  2,
  0,
  4,
  5,
  5,
  3,
  0,
  0,
  5,
  1,
  4,
  1,
  2,
  0,
  5,
  3,
  0,
  5,
  4,
  1,
  5,
  3,
  1,
  4,
  4,
  5,
  2,
  0,
  1,
  0,
  3,
  1,
  5]]

In [32]:
random.seed(12345)
población_inicial = caja_de_herramientas.población()
población_final, registro = algorithms.eaSimple(población_inicial,
                                                caja_de_herramientas,
                                                cxpb=0.5,  # Probabilidad de cruzamiento
                                                mutpb=0.3,  # Probabilidad de mutación
                                                ngen=20,  # Número de generaciones
                                                verbose=False)

for individuo in población_final:
    print(individuo, caja_de_herramientas.evaluate(individuo))

print(registro)

AttributeError: 'Toolbox' object has no attribute 'evaluate'

In [ ]:
# Vamos a proporner las mejores soluciones para resolver el problema 
# planteado.

In [ ]:
print('Las tres mejores soluciones encontradas han sido:')
for individuo in salón_fama:
    print('Individuo: P1={1}, P2={2}; Fitness: {0}'.format(
        individuo.fitness.values[0], *fenotipo(individuo)))